In [1]:
import pandas as pd
# import re
from datetime import datetime
import string

In [2]:
# Correção para linhas que começam com múltiplos espaços

def correctSplit(x):
    splitted_str = list(x)
    if splitted_str[0] == '|':
        splitted_str[0] = ''
    return ''.join(splitted_str)

# Função para separar colunas do df importado

def separar_colunas_df(df):
    # lista alfabetica e.g ['a','b','c'...]
    lista_alfabeto = list(string.ascii_lowercase)
    # Pega a primeira coluna do df (que tem todos os dados) e substitui onde possui mais de um espaço por | 
    df.iloc[:,0] = df.iloc[:,0].replace({'( ){2,} ':'|'},regex=True)
    # corrige a substituição para linhas que começam com espaços
    df.iloc[:,0] = df.iloc[:,0].apply(lambda x: correctSplit(x))
    # corta a partir do separador '|' e define cada pedaço para uma coluna ['a','b'..]
    for i in range (len(df.iloc[:,0][0].split('|'))):
        df[lista_alfabeto[i]] = df.iloc[:,0].apply(lambda x: x.split('|')[i])
    # deleta a coluna que originou as demais colunas    
    df.drop(df.columns[0], axis = 1, inplace = True)
    return df

# veic_2019_02.csv_amostra

In [3]:
colnames = ['index_df', 'x', 'y', 'data-hora', 'data', 'dia', 'mes', 
            'ano', 'tempo', 'hora', 'a',
           'b', 'c', 'd', 'e', 'f', 'placa', 'h', 
            'i', 'j', 'k', 'l' ]

df_veic = pd.read_csv('amostras/veic_2019_02.csv_amostra.txt', sep=",", names=colnames)
df_veic.head(4)

,index_df,x,y,data-hora,data,dia,mes,ano,tempo,hora,...,c,d,e,f,placa,h,i,j,k,l
0,index,n,x,tipo,data,dia,mes,ano,hora,y,...,vel2,k,s,cod,placa,r,t,y,u,i
1,1,501,1,2019-02-01 00:00:07,2019-02-01,01,02,2019,00:00:07,00,...,68.0,2,3.8,bdac468f1,LRV5999,0,0,NaN,NaN,NaN
2,2,502,2,2019-02-01 00:00:09,2019-02-01,01,02,2019,00:00:09,00,...,57.0,2,3.8,bdac469a2,LRW4982,0,0,NaN,NaN,NaN
3,3,502,2,2019-02-01 00:00:17,2019-02-01,01,02,2019,00:00:17,00,...,61.0,2,3.6,bdac46ee2,LQM9654,0,0,NaN,NaN,NaN


# movimento_OCR_JAN_2019.txt_amostra.txt¶

In [31]:
# Nome para coluna inicial
colnames = ['x']

# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df_ocr_raw = pd.read_csv('amostras/movimento_OCR_JAN_2019.txt_amostra.txt', sep='|', names = colnames)

df_ocr_split = separar_colunas_df(df_ocr_raw)
df_ocr_split.head()

,a,b,c,d,e
0,KPV68771AUTOMÓVEL,20190101000000971024,0101201900000003900CET8-5455 0410191112025201...,St R Catete,B-22.93380-43.18294
1,LQR82471AUTOMÓVEL,20190101000001181044,0101201900000103400CET8-5410 0410121122025201...,St Tanque,D-22.97019-43.41377
2,ETP18901AUTOMÓVEL,20190101000001281014,0101201900000104000CET8-5436 0410241111025201...,St Cidade Nova,A-22.93149-43.20888
3,KXL99161AUTOMÓVEL,20190101000001381014,0101201900000102500CET8-5439 0410161111025201...,St Colonia,A-22.92479-43.38907
4,LPU22871AUTOMÓVEL,20190101000001901024,0101201900000104400CET8-5401 0410381122025201...,St Pontal,B-23.01267-43.38900


In [30]:
# testando se existe alguma placa que começa com um número
[i  for i in list(df_ocr_raw.a) if not i[0].isalpha()]

['1AUTOMÓVEL',
 '1AUTOMÓVEL',
 '1AUTOMÓVEL',
 '1AUTOMÓVEL',
 '1AUTOMÓVEL',
 '1AUTOMÓVEL',
 '1CAMINHÃO',
 '3ÔNIBUS',
 '1AUTOMÓVEL',
 '1MOTOCICLETA',
 '1AUTOMÓVEL',
 '1AUTOMÓVEL',
 '1MOTOCICLETA',
 '1AUTOMÓVEL',
 '1AUTOMÓVEL',
 '1AUTOMÓVEL',
 '1AUTOMÓVEL',
 '1MOTOCICLETA',
 '1AUTOMÓVEL',
 '1MOTOCICLETA',
 '1AUTOMÓVEL']

# Tratando

In [32]:
def get_datahora(string):
    date = string[:14]
    try: 
        date = datetime.strptime(date, "%Y%m%d%H%M%S")
    except:
        date = 'erro'
    return date

def get_placa(string):
    placa = string[:7]
    if placa[0].isalpha():
        pass
    else:
        placa = ''
    return placa

def get_latitude(string):
    lat = string[1:10]
    return lat

def get_longitude(string):
    lon = string[10:19]
    return lon

def get_velocidade(string):
    vel = string[14:17]
    try:
        vel = int(vel)
    except:
        vel = 'erro'
    return vel

# Cria o dataframe vazio
df_ocr = pd.DataFrame()

# Popula o dataframe criando cada coluna
df_ocr['CODCET'] = df_ocr_split['c'].apply(lambda x: x.split()[1][:10])
df_ocr['DATAHORA'] = df_ocr_split['b'].apply(lambda x: get_datahora(x))
df_ocr['PLACA'] = df_ocr_split['a'].apply(lambda x: get_placa(x))
df_ocr['VELOCIDADE'] = df_ocr_split['c'].apply(lambda x: get_velocidade(x))
df_ocr['CGCT'] = df_ocr_split['c'].apply(lambda x: x.split()[1][:3])
df_ocr['LATITUDE'] = df_ocr_split['e'].apply(lambda x: get_latitude(x))
df_ocr['LONGITUDE'] = df_ocr_split['e'].apply(lambda x: get_longitude(x))

# Descomentar as próximas linhas caso queira checar cada código separadamente

# df_ocr['CGPT'] = df_ocr_split['c'].apply(lambda x: x.split()[1][3:6])
# df_ocr['CGLG'] = df_ocr_split['c'].apply(lambda x: x.split()[1][6:7])
# df_ocr['CGPI'] = df_ocr_split['c'].apply(lambda x: x.split()[1][7:8])
# df_ocr['CGST'] = df_ocr_split['c'].apply(lambda x: x.split()[1][8:9])
# df_ocr['CGFX'] = df_ocr_split['c'].apply(lambda x: x.split()[1][9:10])

df_ocr.sample(10)

,CODCET,DATAHORA,PLACA,VELOCIDADE,CGCT,LATITUDE,LONGITUDE
62,0410551112,2019-01-01 00:00:26,LQZ5144,30,041,-23.00850,-43.43940
29,0410552121,2019-01-01 00:00:10,KRR1791,31,041,-23.00726,-43.43819
4,0410381122,2019-01-01 00:00:01,LPU2287,44,041,-23.01267,-43.38900
61,0410151112,2019-01-01 00:00:25,KPA6853,33,041,-22.94040,-43.36801
98,0410431112,2019-01-01 00:00:46,,64,041,-22.87778,-43.32760
13,0410421121,2019-01-01 00:00:04,LPS7240,41,041,-22.99197,-43.59319
7,0410021111,2019-01-01 00:00:02,,23,041,-22.79752,-43.19290
5,0410551112,2019-01-01 00:00:02,LKU5609,39,041,-23.00850,-43.43940
83,0410551112,2019-01-01 00:00:36,KYG3450,30,041,-23.00850,-43.43940
2,0410241111,2019-01-01 00:00:01,ETP1890,40,041,-22.93149,-43.20888


# SCP_INTVIA_0182018_201901_amostra.txt¶

In [11]:
# Nome para colunas desconhecidas
colnames = ['x']
# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df_intvia = pd.read_csv('amostras/SCP_INTVIA_0182018_201901_amostra.txt', sep='|', names = colnames)

# Separando as colunas manualmente
df_intvia['x'] = df_intvia.x.replace({'( ){2,} ':'|'},regex=True) # Substituindo 2 ou + espaços pelo caracter "|"
df_intvia['x'] = df_intvia.x.apply(lambda x: correctSplit(x))
df_intvia['veiculo'] = df_intvia.x.apply(lambda x: x.split('|')[0])
df_intvia['a'] = df_intvia.x.apply(lambda x: x.split('|')[1])
df_intvia['b'] = df_intvia.x.apply(lambda x: x.split('|')[2])
df_intvia['c'] = df_intvia.x.apply(lambda x: x.split('|')[3])
df_intvia['d'] = df_intvia.x.apply(lambda x: x.split('|')[4])

# Dropando a coluna x utilizada para separar as outras colunas
df_intvia.drop('x', axis = 1, inplace = True)

df_intvia.head()

,veiculo,a,b,c,d
0,-------1moto,0101201900000215,BRI712,04800911100182018Rua Jardim Botanico,GAVEA
1,LSY84841moto,0101201900000417,BRI712,04800911100182018Rua Jardim Botanico,GAVEA
2,-------1moto,0101201900000714,BRI712,04800911100182018Rua Jardim Botanico,GAVEA
3,-------1moto,0101201900002229,BRI712,04800911100182018Rua Jardim Botanico,GAVEA
4,-------1moto,0101201900004443,BRI712,04800911100182018Rua Jardim Botanico,GAVEA


# SCP_RADAR_RIO_SITRAN_0232015_201903_amostra.txt¶

In [12]:
# Nome para colunas desconhecidas
colnames = ['x']
# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df_sitran = pd.read_csv('amostras/SCP_RADAR_RIO_SITRAN_0232015_201903_amostra.txt', sep='|', names = colnames)
            
# Separando as colunas manualmente
df_sitran['x'] = df_sitran.x.replace({'( ){2,} ':'|'},regex=True) # Substituindo 2 ou + espaços pelo caracter "|"
df_sitran['x'] = df_sitran.x.apply(lambda x: correctSplit(x))
df_sitran['veiculo'] = df_sitran.x.apply(lambda x: x.split('|')[0])
df_sitran['a'] = df_sitran.x.apply(lambda x: x.split('|')[1])
df_sitran['b'] = df_sitran.x.apply(lambda x: x.split('|')[2])
df_sitran['c'] = df_sitran.x.apply(lambda x: x.split('|')[3])
df_sitran['d'] = df_sitran.x.apply(lambda x: x.split('|')[4])

# Dropando a coluna x utilizada para separar as outras colunas
df_sitran.drop('x', axis = 1, inplace = True)

df_sitran.head()

,veiculo,a,b,c,d
0,-------1moto,0103201910263053,BRI1784,04001411100232015RUA ARQUIAS CORDEIRO,ENGENHO NOVO
1,-------1moto,0103201910263141,BRI1784,04001411100232015RUA ARQUIAS CORDEIRO,ENGENHO NOVO
2,-------1moto,0103201910263449,BRI1784,04001411100232015RUA ARQUIAS CORDEIRO,ENGENHO NOVO
3,LQS56581moto,0103201910490948,BRI1784,04001411100232015RUA ARQUIAS CORDEIRO,ENGENHO NOVO
4,FLT20441moto,0103201910491046,BRI1784,04001411100232015RUA ARQUIAS CORDEIRO,ENGENHO NOVO


# SCP_TRANSCARIOCA_0222016_201903_amostra.txt¶

Previamente a minha coluna que continham os dados de latitude e longitude não constava no dataframe, então talvez haja a necessidade de criar uma nova coluna (no meu caso a coluna 'e') para depois conseguirmos tratar esses dados. Achei válido reiterar porque estava me gerando confusão. Vale fazer um check com o arquivo txt para verificar se existe algum dado que talvez se pareça com coordenadas de lat e long.

In [14]:
# Nome para colunas desconhecidas
colnames = ['x']
# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df_transcarioca = pd.read_csv('amostras/SCP_TRANSCARIOCA_0222016_201903_amostra.txt', sep='|', names = colnames)
            
# Separando as colunas manualmente
df_transcarioca['x'] = df_transcarioca.x.replace({'( ){2,} ':'|'},regex=True) # Substituindo 2 ou + espaços pelo caracter "|"
df_transcarioca['x'] = df_transcarioca.x.apply(lambda x: correctSplit(x))
df_transcarioca['veiculo'] = df_transcarioca.x.apply(lambda x: x.split('|')[0])
df_transcarioca['a'] = df_transcarioca.x.apply(lambda x: x.split('|')[1])
df_transcarioca['b'] = df_transcarioca.x.apply(lambda x: x.split('|')[2])
df_transcarioca['c'] = df_transcarioca.x.apply(lambda x: x.split('|')[3])
df_transcarioca['d'] = df_transcarioca.x.apply(lambda x: x.split('|')[4])
df_transcarioca['e'] = df_transcarioca.x.apply(lambda x: x.split('|')[5])

# Dropando a coluna x utilizada para separar as outras colunas
df_transcarioca.drop('x', axis = 1, inplace = True)

df_transcarioca.head(10)

,veiculo,a,b,c,d,e
0,-------2carro de passeio,010320190010319,BRI1813,04400211200222016Av. Ministro E. Romero,IRAJA,"3-22,87280-43,33629"
1,-------1moto,0103201900104424,BRI1813,04400211200222016Av. Ministro E. Romero,IRAJA,"2-22,87280-43,33629"
2,-------2carro de passeio,0103201900104419,BRI1813,04400211200222016Av. Ministro E. Romero,IRAJA,"3-22,87280-43,33629"
3,-------1moto,0103201900104628,BRI1813,04400211200222016Av. Ministro E. Romero,IRAJA,"3-22,87280-43,33629"
4,-------1moto,0103201900104830,BRI1813,04400211200222016Av. Ministro E. Romero,IRAJA,"2-22,87280-43,33629"
5,-------1moto,0103201900105030,BRI1813,04400211200222016Av. Ministro E. Romero,IRAJA,"2-22,87280-43,33629"
6,-------1moto,0103201900105834,BRI1813,04400211200222016Av. Ministro E. Romero,IRAJA,"2-22,87280-43,33629"
7,-------1moto,0103201900110732,BRI1813,04400211200222016Av. Ministro E. Romero,IRAJA,"3-22,87280-43,33629"
8,-------1moto,0103201900110833,BRI1813,04400211200222016Av. Ministro E. Romero,IRAJA,"3-22,87280-43,33629"
9,-------1moto,0103201900111134,BRI1813,04400211200222016Av. Ministro E. Romero,IRAJA,"2-22,87280-43,33629"


In [18]:
def get_datahora(string):
    date = string[:14]
    return datetime.strptime(date, "%d%m%Y%H%M%S")

def get_placa(string):
    placa = string[:7]
    if placa[0].isalpha():
        pass
    else:
        placa = ''
    return placa

def get_latitude(string):
    lat = string[1:4]+"."+string[5:10]
    return lat

def get_longitude(string):
    lon = string[10:13]+"."+string[14:19]
    return lon

df_transcarioca['DATAHORA'] = df_transcarioca['a'].apply(lambda x: get_datahora(x))
df_transcarioca['PLACA'] = df_transcarioca['veiculo'].apply(lambda x: get_placa(x))
df_transcarioca['VELOCIDADE'] = df_transcarioca['a'].str[14:]+".00"
df_transcarioca['LATITUDE'] = df_transcarioca['e'].apply(lambda x: get_latitude(x))
df_transcarioca['LONGITUDE'] = df_transcarioca['e'].apply(lambda x: get_longitude(x))
df_transcarioca['CODCET'] = df_transcarioca['c'].str[:10]
df_transcarioca['CGCT'] = df_transcarioca['c'].str[:3]
df_transcarioca['CGPT'] = df_transcarioca['c'].str[3:6]
df_transcarioca['CGLG'] = df_transcarioca['c'].str[6:7]
df_transcarioca['CGPI'] = df_transcarioca['c'].str[7:8]
df_transcarioca['CGST'] = df_transcarioca['c'].str[8:9]
df_transcarioca['CGFX'] = df_transcarioca['c'].str[9:10]
df_transcarioca.drop(['veiculo', 'a', 'b', 'c', 'd', 'e', 'CGPT', 'CGPI', 'CGST', 'CGFX', 'CGLG'], axis=1, inplace=True)
df_transcarioca.head(20)

,DATAHORA,PLACA,VELOCIDADE,LATITUDE,LONGITUDE,CODCET,CGCT
0,2019-03-01 00:10:31,,9.00,-22.87280,-43.33629,0440021120,044
1,2019-03-01 00:10:44,,24.00,-22.87280,-43.33629,0440021120,044
2,2019-03-01 00:10:44,,19.00,-22.87280,-43.33629,0440021120,044
3,2019-03-01 00:10:46,,28.00,-22.87280,-43.33629,0440021120,044
4,2019-03-01 00:10:48,,30.00,-22.87280,-43.33629,0440021120,044
5,2019-03-01 00:10:50,,30.00,-22.87280,-43.33629,0440021120,044
6,2019-03-01 00:10:58,,34.00,-22.87280,-43.33629,0440021120,044
7,2019-03-01 00:11:07,,32.00,-22.87280,-43.33629,0440021120,044
8,2019-03-01 00:11:08,,33.00,-22.87280,-43.33629,0440021120,044
9,2019-03-01 00:11:11,,34.00,-22.87280,-43.33629,0440021120,044


# SPLICE_007201820190205.txt_amostra.txt¶

In [15]:
# Nome para colunas desconhecidas
colnames = ['x']
# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df_splice = pd.read_csv('amostras/SPLICE_0072018_20190205_.txt_amostra.txt', sep='|', names = colnames)
            
# Separando as colunas manualmente
df_splice['x'] = df_splice.x.replace({'( ){2,} ':'|'},regex=True) # Substituindo 2 ou + espaços pelo caracter "|"
df_splice['x'] = df_splice.x.apply(lambda x: correctSplit(x))
df_splice['veiculo'] = df_splice.x.apply(lambda x: x.split('|')[0])
df_splice['a'] = df_splice.x.apply(lambda x: x.split('|')[1])
df_splice['b'] = df_splice.x.apply(lambda x: x.split('|')[2])
df_splice['c'] = df_splice.x.apply(lambda x: x.split('|')[3])

# Dropando a coluna x utilizada para separar as outras colunas
df_splice.drop('x', axis = 1, inplace = True)

df_splice.sample(5)

,veiculo,a,b,c
13,1MOTO,2019010100004705300RD3399,04505612120072018AV PRES ANTONIO CARLOS PX ED ...,2-22.90751-43.17307
3,1MOTO,2019010100002203900RD3414,04506811110072018R VISCONDE DE PIRAJA PX175,Copacabana
10,1MOTO,2019010100003605500RD3386,04504711120072018AV GAL SAN MARTIN PX 801,Sao Conrado
99,1MOTO,2019010100082605000RD3394,04505211220072018R ESTACIO SA PX HOSPITAL POLI...,Centro
76,1MOTO,2019010100053003800RD3381,00450541110072018R HUMAITA PX 144 - ST JD BOTA...,Jardim Botanico


# FISCALTECH_0042016_06_2019.txt_amostra.txt

In [16]:
# Nome para colunas desconhecidas
colnames = ['x']
# Usando separador '|' apenas para conseguir importar e depois tentar tratar
df_fiscaltech = pd.read_csv('amostras/FISCALTECH_0042016_06_2019.txt_amostra.txt', sep='|', names = colnames)
            
# Separando as colunas manualmente
df_fiscaltech['x'] = df_fiscaltech.x.replace({'( ){3,} ':'|'},regex=True) # Substituindo 2 ou + espaços pelo caracter "|"
df_fiscaltech['x'] = df_fiscaltech.x.apply(lambda x: correctSplit(x))
df_fiscaltech['a'] = df_fiscaltech.x.apply(lambda x: x.split('|')[0])
df_fiscaltech['b0'] = df_fiscaltech.x.apply(lambda x: x.split('|')[1])
df_fiscaltech['b'] = df_fiscaltech.b0.apply(lambda x: x.split(' ')[0])
df_fiscaltech['c'] = df_fiscaltech.b0.apply(lambda x: ' '.join(list(x.split(' ')[1:])))
df_fiscaltech['d'] = df_fiscaltech.x.apply(lambda x: x.split('|')[2])
# df_fiscaltech['c'] = df_fiscaltech.x.apply(lambda x: x.split('|')[3])

# Dropando a coluna x utilizada para separar as outras colunas
df_fiscaltech.drop(['x','b0'], axis = 1, inplace = True)

df_fiscaltech.sample(5)

,a,b,c,d
64,1,0106201903195504000FSCII2180,04200811240042016Av Americas Px1155 PCent-St ...,4-23003226-43324684
31,1,0106201903370806900FSCII2160,04200412120042016Av Americas Px2250 PLat-St S...,2-23000029-43339268
19,3,0106201906192700900FSCII2181,04200811110042016Av Americas Px900 PCent-St S...,1 -2300303 -4332414
45,3,0106201903480506000FSCII2176,04200712220042016Av Americas Px2151 PLat-St S...,2-23001678-43328644
88,1,0106201903324001400FSCII2175,04200611140042016Av Americas Px2000 PCent-St ...,4-23000409-43334161
